# Process Framework

## Overview

Two process steps will and one supporting process step will be defined: 

- `retrieveWinnerProcessStep`: Defines the functionality to retrieve the winner of the sport event. 
- `retrieveScoreProcessStep`: Defines the functionality to retrieve the score of the sport event.
- `storeResultProcessStep`: Acts as supporting process step which can store winner and score and will finally show the result of the process flow.

All three steps are defined as classes with class functions, uses Semantic Kernel and the function calling possibilities of the deployed OpenAI GPT-4o model to identify and execute local functions.

![](../media/img/Process_Overview.png)



```csharp
processBuilder
    .OnInputEvent("RetrieveSportEventResult")
    .SendEventTo(new (retrieveWinnerProcessStep));

retrieveWinnerProcessStep
    .OnFunctionResult()
    .SendEventTo(new (retrieveScoreProcessStep))
    .SendEventTo(new (storeResultProcessStep, functionName: "StoreResult"));

retrieveScoreProcessStep
    .OnFunctionResult()
    .SendEventTo(new (storeResultProcessStep, functionName: "DisplayResults"));
```






## Environment

An Azure OpenAI GPT-4 instance will be used by Semantic Kernel and it's current Agent framework to identify which local functions need to be executed. The necessary connection information is imported from the ***config.env*** file in the [../config/ folder](../config). You can use the provided [Azure CLI Powershell script](../setup/setup.azcli) to create the necessary instance.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.38.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.38.0-alpha"
#r "nuget: DotNetEnv, 3.1.1"

using System.ClientModel; 
using System.ComponentModel;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Process;
using DotNetEnv;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

public class Configuration {
    public string ApiKey { get; set; } = Env.GetString("SK_OPENAI_APIKEY");
    public string Endpoint { get; set; } = Env.GetString("SK_OPENAI_ENDPOINT");
    public string ChatCompletionDeploymentDefault { get; set; } = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");
}

Console.WriteLine($"Configuration loaded...");

## Define Process Step Functionality

### RetrieveWinnerProcessStep

Semantic Kernel and it's plug-in architecture which uses the function calling API of a LLM or SLM is used to identify and execute local functions to identify the winner of the sport event. 

The function(s) provided in the type `RetrieveWinner` are provided to Semantic Kernel to identify and execute the local function `public string GetSportEventWinner()` with the correct parameters.  

```csharp
        kernel.Plugins.Clear();
        kernel.Plugins.AddFromType<RetrieveWinner>("RetrieveWinner");
```


In [ ]:
#pragma warning disable SKEXP0080

public class RetrieveWinnerProcessStep : KernelProcessStep
{
    [KernelFunction]
    public async Task<string> QueryForWinner(Kernel kernel, string sportEventName)
    {
        Console.WriteLine($"\n'QueryForWinner({sportEventName}) called...");
        
        IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

        string prompt = @$"
            Who won the {sportEventName}? 
            Just return the name of the winning team.
        ";
        ChatHistory chatHistory = new ChatHistory();
        chatHistory.AddUserMessage(prompt);
        
        kernel.Plugins.Clear();
        kernel.Plugins.AddFromType<RetrieveWinner>("RetrieveWinner");

        KernelFunction getSportEventWinner = kernel.Plugins.GetFunction("RetrieveWinner", "get_sport_event_winner");
        PromptExecutionSettings promptExecutionSettings = new PromptExecutionSettings() {
            FunctionChoiceBehavior = FunctionChoiceBehavior.Required(
                new List<KernelFunction>() { getSportEventWinner }  
            )
        };
        promptExecutionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Auto();
        
        Console.WriteLine($"\t'ChatCompletionService' called...");
        ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
            chatHistory: chatHistory, 
            executionSettings: promptExecutionSettings,
            kernel: kernel
        );

        Console.WriteLine($"\t'ChatCompletionService' returned {chatMessageContent.Content}...");
        return $"Winner: {chatMessageContent.Content ?? ""} - Event: {sportEventName}";
    }

    public class RetrieveWinner : KernelProcessStep
    {
        [KernelFunction("get_sport_event_winner")]
        [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
        public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
        {
            
            Console.WriteLine($"\tPlugin: 'GetSportEventWinner({sportEventName}, {sportEventYear})' called...");
            Console.WriteLine($"\tPlugin: 'GetSportEventWinner({sportEventName}, {sportEventYear})' returning 'Munich Flying Dolphins'...");
            
            // Implement the logic to get the winner of the sport event.
            return "Munich Flying Dolphins";
        }
    }
 
}

Console.WriteLine($"'RetrieveWinnerProcessStep' defined...");


### RetrieveScoreProcessStep

Semantic Kernel and it's plug-in architecture which uses the function calling API of a LLM or SLM is used to identify and execute local functions to identify the winner of the sport event. 

The function(s) provided in the type `RetrieveScore` are provided to Semantic Kernel to identify and execute the local function `public string GetSportEventWinner()` with the correct parameters.  

```csharp
        kernel.Plugins.Clear();
        kernel.Plugins.AddFromType<RetrieveScore>("RetrieveScore");
```


In [ ]:
#pragma warning disable SKEXP0080

public class RetrieveScoreProcessStep : KernelProcessStep
{

    [KernelFunction]
    public async Task<string> QueryForScore(Kernel kernel, string sportEventName)
    {

        Console.WriteLine($"\n'QueryForScore({sportEventName}) called...");
        
        IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

        string prompt = @$"
            	What was the score of the {sportEventName}? 
                Just return the score."
        ;
        ChatHistory chatHistory = new ChatHistory();
        chatHistory.AddUserMessage(prompt);
        
        kernel.Plugins.Clear();
        kernel.Plugins.AddFromType<RetrieveScore>("RetrieveScore");

        KernelFunction getSportEventScore = kernel.Plugins.GetFunction("RetrieveScore", "get_sport_event_score");
        PromptExecutionSettings promptExecutionSettings = new PromptExecutionSettings() {
            FunctionChoiceBehavior = FunctionChoiceBehavior.Required(
                new List<KernelFunction>() { getSportEventScore }  
            )
        };
        promptExecutionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Required();
        
        Console.WriteLine($"\t'ChatCompletionService' called...");       
        ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
            chatHistory: chatHistory, 
            executionSettings: promptExecutionSettings,
            kernel: kernel
        );
        Console.WriteLine($"\t'ChatCompletionService' returned: {chatMessageContent.Content}...");
        
        return chatMessageContent.Content ?? "";
    }


    public class RetrieveScore
    {
        [KernelFunction("get_sport_event_score")]
        //Description is just necessary if function name is cryptic and not self-explanatory
        [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
        public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
        {
            Console.WriteLine($"\tPlugin: 'GetSportEventScore({sportEventName}, {sportEventYear})' called...");
            Console.WriteLine($"\tPlugin: 'GetSportEventScore({sportEventName}, {sportEventYear})' returning '24:1'...");
            // Implement the logic to get the result of the sport event.
            return "24:1";
        }
    }

}  

Console.WriteLine($"'RetrieveScoreProcessStep' definied...");



### StoreResultHelper

A supportive process step which is used to store information from other process steps.

In [ ]:
#pragma warning disable SKEXP0080

public class StoreResultHelper : KernelProcessStep
{
    static List<string> _storage = new List<string>();

    [KernelFunction]
    public async Task StoreResult(string result)
    {
        await Task.Run( () => _storage.Add(result));
    }

    [KernelFunction]
    public async Task DisplayResults(string result)
    {
        Console.WriteLine($"\nProcess results:");
        await Task.Run( () => {
            foreach (string result in _storage)
            {
                Console.WriteLine($"\tStored result: {result}");
            }
            Console.WriteLine($"\tStored result: {result}");
            _storage.Clear();
        });
    }
}

Console.WriteLine($"'StoreResultHelper' defined...");

## Process Definition

The three process steps are 'wired' together using the Semantic Kernel Process Framework:

- `retrieveWinnerProcessStep` will get called when an 'event' `retrieveSportEventResult` is triggered within the process flow. 
- `retrieveScoreProcessStep` will get an 'event' called from `retrieveWinnerProcessStep`
- `storeResultProcessStep` will get an 'event' both from `retrieveSportEventResult` and `retrieveScoreProcessStep`. The 'event' from `retrieveScoreProcessStep` will be delivered to the function `DisplayResults()` which will trigger displaying the final result of the process flow.


In [ ]:
#pragma warning disable SKEXP0080

// Create the process builder
ProcessBuilder processBuilder = new("RetrieveSportEventResult");

// Add the steps
ProcessStepBuilder retrieveWinnerProcessStep = processBuilder.AddStepFromType<RetrieveWinnerProcessStep>();
ProcessStepBuilder retrieveScoreProcessStep = processBuilder.AddStepFromType<RetrieveScoreProcessStep>();
ProcessStepBuilder storeResultProcessStep = processBuilder.AddStepFromType<StoreResultHelper>();

processBuilder
    .OnInputEvent("RetrieveSportEventResult")
    .SendEventTo(new (retrieveWinnerProcessStep));

retrieveWinnerProcessStep
    .OnFunctionResult()
    .SendEventTo(new (retrieveScoreProcessStep))
    .SendEventTo(new (storeResultProcessStep, functionName: "StoreResult"));

retrieveScoreProcessStep
    .OnFunctionResult()
    .SendEventTo(new (storeResultProcessStep, functionName: "DisplayResults"));


KernelProcess kernelProcess = processBuilder.Build();

Console.WriteLine($"Process defined ...");

## Kernel / Process Start

A Semantic Kernel instance `kernel` will defined which hosts and executes the defined process flow.

Because the process flow and necessary input / output data and format is known just the necessary information to start the process flow is needed. Therefore 'just' the name of the sport event ***Super Sports Championship 2025*** needs to be provided.

In [ ]:
#pragma warning disable SKEXP0080

Configuration _configuration = new Configuration();
IKernelBuilder kernelBuilder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        apiKey: _configuration.ApiKey, 
        endpoint: _configuration.Endpoint, 
        deploymentName: _configuration.ChatCompletionDeploymentDefault
    );
kernelBuilder.Services.AddSingleton<Configuration>(_configuration);
Kernel kernel = kernelBuilder.Build();

KernelProcessContext kernelProcessContext = await kernelProcess.StartAsync(
    kernel, 
    new KernelProcessEvent { 
        Id = "RetrieveSportEventResult", 
        Data = "Super Sports Championship 2025" 
    }
);

Console.WriteLine($"\nProcess finished...");

Execution details from the specific steps, with the final result of the process flow are shown. 